In [2]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl import load_workbook

from collections import OrderedDict
from operator import getitem
import itertools


In [3]:
# Load Spend Overview data
raw_data_overview = pd.read_excel("input_spend overview.xlsx",'Raw Data')
raw_data_overview.replace(np.nan,'',inplace=True)

raw_data_overview['ID'] = 'Spend Category: ' + raw_data_overview['Spend Category'] + '; ' + 'Origin: '+ raw_data_overview['Origin Name'] + '; '+ 'Destination: '+raw_data_overview['Destination Name'] + '; ' + 'Supplier: '+raw_data_overview['Supplier'] + '; '+ 'Container: '+ raw_data_overview['Container Type']
raw_data_overview ['Corridor'] = raw_data_overview['Origin Name'] + '-' + raw_data_overview['Destination Name']


#Drop rows with 0 coverage
raw_data_overview = raw_data_overview[raw_data_overview['Coverage Percent'] != 0]

In [4]:
# Load ALL data
raw_data_all = pd.read_excel("input_all spend.xlsx",'Raw Data')

raw_data_all.replace(np.nan,'',inplace=True)

raw_data_all ['ID'] =  'Origin: '+ raw_data_all['Origin Name'] + '; '+ 'Destination: '+raw_data_all['Destination Name'] + '; ' + 'Supplier: '+raw_data_all['Supplier'] + '; '+ 'Container: '+ raw_data_all['Container Type'] + '; '+ 'Contract Number: '+ raw_data_all['Contract Number']
raw_data_all ['Lane'] = raw_data_all['Origin Name'] + '-' + raw_data_all['Destination Name'] + ', ' + raw_data_all['Container Type'] + ', ' + raw_data_all['Supplier']
#raw_data_dict = dict(raw_data)
#raw_data_dict = dict(raw_data)

#Drop rows with 0 coverage
raw_data_all = raw_data_all[raw_data_all['Rate Quality'] != 'No coverage']

#Compute cost avoidance/saving pot
#raw_data_all[['eq_no', 'eq_type']] = raw_data_all["Container Type"].apply(lambda x: pd.Series(str(x).split("'")))


In [5]:
raw_data_all.head()

,Day,Origin Name,Origin Code,Destination Name,Destination Code,Supplier,Container Type,Contract Number,Valid From,Valid To,...,Saving Potential (VS Market Midlow),Market Midhigh,Saving Potential (VS Market Midhigh),Rate Quality,Aggregation Level,Used Origin,Used Destination,THC Methodology,ID,Lane
0,2023-04-06,Nansha,CNNSA,Dunkerque,FRDKK,Carrier/FF 02,40’ High Cube,Update 2023-01-01,2023-01-01,2023-06-30,...,3713769.0,1800.0,3546369.0,Less than 10,1,China South Main,Dunkerque,NONE,Origin: Nansha; Destination: Dunkerque; Suppli...,"Nansha-Dunkerque, 40’ High Cube, Carrier/FF 02"
1,2023-04-06,Shanghai,CNSGH,Antwerpen,BEANR,Carrier/FF 02,40’ Standard Dry,Update 2023-01-01,2023-01-01,2023-06-30,...,2920393.0,2109.0,2500666.0,Less than 10,0,Shanghai,Antwerpen,NONE,Origin: Shanghai; Destination: Antwerpen; Supp...,"Shanghai-Antwerpen, 40’ Standard Dry, Carrier/..."
2,2023-04-06,Ningbo,CNNBO,Le Havre,FRLEH,Carrier/FF 02,40’ High Cube,Update 2023-01-01,2023-01-01,2023-06-30,...,2706312.0,1650.0,2619092.0,Less than 10,0,Ningbo,Le Havre,NONE,Origin: Ningbo; Destination: Le Havre; Supplie...,"Ningbo-Le Havre, 40’ High Cube, Carrier/FF 02"
3,2023-04-06,Ningbo,CNNBO,Antwerpen,BEANR,Carrier/FF 02,40’ High Cube,Update 2023-01-01,2023-01-01,2023-06-30,...,2560896.0,4688.0,1006848.0,10 +,1,China East Main,Antwerpen,NONE,Origin: Ningbo; Destination: Antwerpen; Suppli...,"Ningbo-Antwerpen, 40’ High Cube, Carrier/FF 02"
4,2023-04-06,Ningbo,CNNBO,Dunkerque,FRDKK,Carrier/FF 02,40’ Standard Dry,Update 2023-01-01,2023-01-01,2023-06-30,...,1765643.0,1600.0,1659343.0,10 +,1,China East Main,Dunkerque,NONE,Origin: Ningbo; Destination: Dunkerque; Suppli...,"Ningbo-Dunkerque, 40’ Standard Dry, Carrier/FF 02"


In [6]:
#raw_data_all ['Saving Potential/Cost Avoidance Towards Market Average'] = raw_data_all ['My Price'] - raw_data_all ['Market Average']

In [7]:
#setup ovwerview dict
dict_overview = {}
for row in raw_data_overview.iterrows():
    id_ = row[1]['ID']
    col_name_list = ['Corridor','Supplier','Origin Name','Destination Name','Volume (TEU)', 'My Spend', 'Coverage Percent', 'My Covered Spend (in $USD)', 'Market Average Spend', 'Market Low Spend', 'Market High Spend', 'My Covered Spend Performance Relative to Market Average Spend (in %)', '$USD Saving Potential Towards Market Average', '$USD Saving Potential Towards Market Low', '$USD Saving Potential Towards Market High']
    for col in col_name_list:
        item = row[1][col]
        try:
            dict_overview[id_][col] = item
        except KeyError:
            inner_dict = {}
            inner_dict[col] = item
            dict_overview[id_] = inner_dict
    #print(id_)
    #dict_ [id_] = row

In [8]:
#Setup all dict
dict_all = {}
for row in raw_data_all.iterrows():
    id_ = row[1]['ID']
    col_name_list = ['Lane','Supplier','Origin Name','Destination Name','Volume (TEU)', 'My Price', 'Market Average', 'Market Low', 'Market High', 'My Spend', 'Saving Potential (VS Market Average)','Saving Potential (VS Market Low)','Saving Potential (VS Market High)','THC Methodology']
    for col in col_name_list:
        item = row[1][col]
        try:
            dict_all[id_][col] = item
        except KeyError:
            inner_dict = {}
            inner_dict[col] = item
            dict_all[id_] = inner_dict
    #print(id_)
    #dict_ [id_] = row

In [9]:
#Load workbook that is being filled with data
workbook = load_workbook(filename="230320_Excel for Value Assessment Presentation graphs - Copy.xlsx")
sheet = workbook.active

In [10]:
#POPULATE OVERVIEW STUFF
cell_position = 0
#First and second chart
#TOTAL PERFORM AND SAVING POTENTIAL
for key in dict_overview:
    if "Total" in key:
        sheet["C5"].value = float(dict_overview[key]["Market Low Spend"])/1000000
        sheet["C6"].value = float(dict_overview[key]["Market Average Spend"]) /1000000
        sheet["C7"].value = float(dict_overview[key]["Market High Spend"]) /1000000
        sheet["C8"].value = float(dict_overview[key]["My Covered Spend (in $USD)"]) /1000000
        
        sheet["D5"].value = float(dict_overview[key]["My Covered Spend (in $USD)"]) /1000000
        sheet["D6"].value = float(dict_overview[key]["My Covered Spend (in $USD)"]) /1000000
        sheet["D7"].value = float(dict_overview[key]["My Covered Spend (in $USD)"]) /1000000
        sheet["D8"].value = float(dict_overview[key]["My Covered Spend (in $USD)"]) /1000000
        
        sheet["H7"].value = float(dict_overview[key]["$USD Saving Potential Towards Market Low"]) /1000000
        sheet["H6"].value = float(dict_overview[key]["$USD Saving Potential Towards Market Average"]) /1000000
        sheet["H5"].value =float( dict_overview[key]["$USD Saving Potential Towards Market High"]) /1000000
        

#Corridor Perf
counter = 0
for key in dict_overview:
    if dict_overview[key]['Origin Name'] != '' and dict_overview[key]['Destination Name'] != '':
        
        #if counter >= 5:
            #sheet.insert_rows[21+counter]
        
        sheet["J"+str(6+counter)].value = dict_overview[key]['Corridor']
        sheet["K"+str(6+counter)].value = dict_overview[key]['My Covered Spend Performance Relative to Market Average Spend (in %)']/100
        sheet["L"+str(6+counter)].value = float(dict_overview[key]['$USD Saving Potential Towards Market Average']) /1000000
        
        sheet["R"+str(6+counter)].value = dict_overview[key]['Corridor']
        sheet["S"+str(6+counter)].value = dict_overview[key]['My Covered Spend Performance Relative to Market Average Spend (in %)']/100
        sheet["T"+str(6+counter)].value = float(dict_overview[key]['$USD Saving Potential Towards Market Average']) /1000000
        
        sheet["W"+str(6+counter)].value = dict_overview[key]['Corridor']
        sheet["X"+str(6+counter)].value = dict_overview[key]['My Covered Spend Performance Relative to Market Average Spend (in %)']/100
        sheet["Y"+str(6+counter)].value = float(dict_overview[key]['$USD Saving Potential Towards Market Average']) /1000000
        counter = counter + 1
        print(key)    

#Supplier Perf
counter = 0
for key in dict_overview:
    if dict_overview[key]['Supplier'] != '':
        sheet["AD"+str(5+counter)].value = dict_overview[key]['Supplier']
        sheet["AE"+str(5+counter)].value = dict_overview[key]['Volume (TEU)']/dict_overview['Spend Category: Total; Origin: ; Destination: ; Supplier: ; Container: ']['Volume (TEU)']
        sheet["AF"+str(5+counter)].value = float(dict_overview[key]['$USD Saving Potential Towards Market Average'])/dict_overview['Spend Category: Total; Origin: ; Destination: ; Supplier: ; Container: ']['$USD Saving Potential Towards Market Average']
        counter = counter + 1
        

        
        

Spend Category: By corridor; Origin: Asia; Destination: Europe; Supplier: ; Container: 
Spend Category: By corridor; Origin: Asia; Destination: Latin America; Supplier: ; Container: 
Spend Category: By corridor; Origin: Europe; Destination: Africa; Supplier: ; Container: 
Spend Category: By corridor; Origin: Asia; Destination: Africa; Supplier: ; Container: 
Spend Category: By corridor; Origin: Europe; Destination: Europe; Supplier: ; Container: 
Spend Category: By corridor; Origin: Europe; Destination: Latin America; Supplier: ; Container: 
Spend Category: By corridor; Origin: Latin America; Destination: Asia; Supplier: ; Container: 


In [11]:
#POPULATE ALL STUFF

dict_all = OrderedDict(sorted(dict_all.items(),
       key = lambda x: getitem(x[1], 'Volume (TEU)'),reverse=True))

dict_all_ten = dict(itertools.islice(dict_all.items(), 10))

counter = 0
for key in dict_all_ten:
    sheet["AK"+str(5+counter)].value = dict_all_ten[key]['Lane']
    sheet["AL"+str(5+counter)].value = dict_all_ten[key]['Volume (TEU)']
    sheet["AM"+str(5+counter)].value = dict_all_ten[key]['Saving Potential (VS Market Average)'] / 1000000
    counter = counter + 1

    
workbook.save(filename="Output_file.xlsx") 